# 1. [40]Language and Topic models

A common suggestion to users for coming up with good queries is to think of words that would likely appear in a relevant document, and to use those words as the query. The language modeling approach to IR directly models this idea: a document is a good match to a query if the document model is likely to generate the query, which will in turn happen if the document contains the query words often. 

You will score documents with respect to user query using language models and also get some experience with topic modelling.

## 1.0. [5]Loading data

We use the dataset we already used once - [this topic-modeling dataset](https://code.google.com/archive/p/topic-modeling-tool/downloads) ([or from github](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/topic-modelling.zip)).

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

# TODO: read the dataset

def read_dataset(file_path):
    docs = []
    with open(file_path) as fp:
        for cnt, line in enumerate(fp):
            docs.append(word_tokenize(line))
    return docs

drivePath = "/content/drive/MyDrive/AIR/"

fuel_data = read_dataset(drivePath + "testdata_news_fuel_845docs.txt")
brain_inj_data = read_dataset(drivePath + "testdata_braininjury_10000docs.txt")
economy_data = read_dataset(drivePath + "testdata_news_economy_2073docs.txt")
music_data = read_dataset(drivePath + "testdata_news_music_2084docs.txt")

all_data = []
all_data.extend(fuel_data)
fuelMax = len(all_data)
all_data.extend(brain_inj_data)
brainMax = len(all_data)
all_data.extend(economy_data)
economyMax = len(all_data)
all_data.extend(music_data)
musicMax = len(all_data)

print("# of documents", len(all_data))
assert len(all_data) == 15002

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
# of documents 15002


## 1.1. [35]Ranking Using Language Models
Our goal is to rank documents by $P(d|q)$, where the probability of a document is interpreted as the likelihood that it is relevant to the query. 

Using Bayes rule: $P(d|q) = \frac{P(q|d)P(d)}{P(q)}$

$P(q)$ is the same for all documents, and so can be ignored. The prior probability of a document $P(d)$ is often treated as uniform across all $d$'s and so it can also be ignored. What does it mean? 

It means that computing $P(q|d)$ for different documents we can compare how relevant are they to the query. How can we estimate $P(q|d)$?

$P(q|d)$ can be estimated as:

![](https://i.imgur.com/BEIMAC1.png)

where $M_d$ is the language model of document $d$, $tf_{t,d}$ is the term frequency of term $t$ in document $d$, and $L_d$ is the number of tokens in document $d$. That is, we just count up how often each word occurred, and divide by the total number of words in the document $d$.

### 1.1.1. [5]Build TDM (or DTM)

The first thing we need to do is to build a term-document matrix for tour dataset.

In [17]:
from collections import Counter
# TODO: build term-document matrix for the dataset
terms = []
tdm = dict()

for doc_id, doc in enumerate(all_data):
    tdm[doc_id] = dict(Counter(doc))
    terms.extend(doc)

terms = list(set(terms))

### 1.1.2. [15]Smoothing

Now, you need to implement the abovementioned logic in the `lm_rank_documents` function below. Do you see any potential problems?

Yes, data sparsity - we don't expect to meet each term in each doc, so, in most cases, we will get zero scores, which is not what we really want.

The solution is smooting.

One option is *additive smoothing* - adding a small number (0 to 1) to the observed counts and renormalizing to give a probability distribution.

Another option is called [Jelinek-Mercer smoothing](http://mlwiki.org/index.php/Smoothing_for_Language_Models#Jelinek-Mercer_Smoothing) - a simple idea that works well in practice is to use a mixture between a document-specific distribution and distribution estimated from the entire collection:

![](https://i.imgur.com/8Qv41Wp.png)

where 0 < λ < 1 and $M_c$ is a language model built from the entire document collection.

Refer to [*Chapter 12*](https://nlp.stanford.edu/IR-book/html/htmledition/language-models-for-information-retrieval-1.html) for the detailed explanation.


You are going to apply both in your `lm_rank_documents` function. This function takes TDM or DTM as an input, and ranks all documents "building" a language model for each document, returning relative probabilities of query being generated by a document as a document's score.

In [26]:
words = []

for ar in all_data:
  for word in ar:
    words.append(word)

totalFreq = Counter(words)

In [30]:
import numpy as np


def lm_rank_documents(query, tdm, terms_list, smoothing='additive', param=0.001):
    # TODO: score each document in tdm using this document's language model
    # implement two types of smoothing. Looks up term frequencies in tdm
    # return document scores in a convenient form
    # param is alpha for additive / lambda for jelinek-mercer
    scores = []
    # i = 0
    scoredicts = dict()
    for doc_id in tdm:
        val = tdm[doc_id]
        lendoc = len(all_data[doc_id])
        # if i < 5:
        for query_word in query:
          query_freq = query[query_word]         
          term_freq = 0
          if doc_id not in scoredicts.keys():
            scoredicts[doc_id] = 0
          if query_word in val:
            term_freq = 1
            
            if smoothing == "additive":
              v1 = (totalFreq[query_word] * (1 - param) / len(totalFreq)) 
              v2 = ((term_freq * param) / lendoc)
              scoredicts[doc_id] += v1 + v2
            
            if smoothing == "jelinek-mercer":
              v1 = (lendoc + (lendoc * param))
              v2 = (param + term_freq)
              scoredicts[doc_id] +=  v1 + v2

      # i+=1

    scores = list(scoredicts.items())
    sortedscore = sorted(scores, key = lambda x: x[1], reverse=True)
    return sortedscore[:5]

### 1.1.3. [15]Testing

Check if this type of ranking gives meaningful results. For each query output document `category`, `doc_id`, `score`, and the *beginning* of the document, as it is shown below. Analyze if categories and contents match the queries. 

In [32]:
def querycategory(id):
  querytype = "" 
  if id < fuelMax:
    querytype = "Fuel"
  elif id < brainMax:
    querytype =  "Brain"
  elif id < economyMax:
    querytype = "Economy"
  elif id < musicMax:
    querytype = "Music"
  return querytype


def process_query(raw_query):
    # TODO: process user query and print search results including document category, id, score, and some part of it
    rank = lm_rank_documents(Counter(raw_query.split()), tdm, terms) # , smoothing='jelinek-mercer'
    print("user query:", raw_query)
    print("\nsearch results:")
    for doc_id, score in rank:
        document = all_data[doc_id]
        topic = querycategory(doc_id)
        print(topic, doc_id ,  score)
        print(" ".join(document)[:87], "...")
        # print(document)
    

user_queries = ["piano concert", "symptoms of head trauma", "wall street journal"]
for q in user_queries:
    process_query(q)
    print("\n")

user query: piano concert
 
search results:
music 13330 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
economy 11335 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
music 12926 0.011382499792705511
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorica...
music 14390 0.010661589922122
hailed los angeles brightest flower its flashiest ship sail its keenest architectural triumph perhap...
music 13818 0.010549141787975117
everything was finished sept the super bowl logo would reflection new orleans featuring streetcar an...
 
 
user query: symptoms of head trauma
 
search results:
brain_injury 7319 0.06022877378376099
the direct economic burden blunt and penetrating trauma managed care population background although ...
brain_injury 6987 0.05854539395767944
history reported head trauma sampl

Sample results can look like this (if collapsed, click on 3 dots):

```
user query: piano concert

search results:
music 13330 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
economy 11335 0.012384164490679759
atlanta prominent midtown intersection one step closer becoming major cultural landmark the atlanta ...
music 12926 0.011382499792705511
felt like was going church marry guy never met said the jazz violinist regina carter the metaphorica...
music 14390 0.010661589922122
hailed los angeles brightest flower its flashiest ship sail its keenest architectural triumph perhap...
music 13818 0.010549141787975117
everything was finished sept the super bowl logo would reflection new orleans featuring streetcar an...


user query: symptoms of head trauma

search results:
brain_injury 7319 0.06022877378376099
the direct economic burden blunt and penetrating trauma managed care population background although ...
brain_injury 6987 0.05854539395767944
history reported head trauma sample women substance abuse treatment objectives determine the prevale...
brain_injury 5257 0.05760140208255336
violent head trauma china report cases background the occurrence violent trauma has recently increas...
brain_injury 1536 0.055365767080148634
mild head trauma and chronic headaches returning soldiers objective determine the incidence and type...
brain_injury 8874 0.05379997937839304
maxillofacial trauma major trauma patients background trauma has been identified major public health...


user query: wall street journal

search results:
economy 11294 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
economy 11295 0.027288833622119528
these business stories for release tuesday january are moving today clients the new york times news ...
music 14641 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
music 14640 0.026716049665405375
these feature stories are moving today clients the new york times news service stories are for relea...
economy 11297 0.025763725974814314
these feature stories are moving today clients the new york times news service stories are for relea...
```

## 1.2. [+10/100] Bonus - topic modeling

Now let's use *Latent Dirichlet Allocation* to identify topics in this collection and check if they match the original topics (fuel, economy, etc.). Go through the tutorial [here](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) and apply the ideas there to our dataset. 

In [ ]:
# TODO: apply LDA to our dataset and output the resulting categories 
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

bigList = list()
for id, doc in enumerate(all_data):
  # if id < 5:
    converted = " ".join(doc)
    bigList.append(converted)

# print(bigList)

count_vectorizer = CountVectorizer()
count_data = count_vectorizer.fit_transform(bigList)

def topics(model, vectorizer, n=10):
    names = vectorizer.get_feature_names()
    id = 0
    for topic in model.components_[:4]:
        topics = ""
        le = n + 5
        arr = topic.argsort()
        for j in arr[: -le :-1]:
          topics += names[j] + " "
        print("Topic #"+ str(id)+ ":")
        print(topics[:87], "...\n")
        id += 1

# LDA model
lda = LDA(n_components=4, n_jobs=-1)
lda.fit(count_data)

topics(lda, count_vectorizer)

Topic #0:
the and that for with was said are have but from has new this  ...

Topic #1:
the and for with that are was patients from were injury who said new  ...

Topic #2:
the and that for with was his from this were they new are said  ...

Topic #3:
the and with for was patients from were that are but this not his  ...



We expect to see something like this (if collapsed, click on 3 dots):

```
Topic #0:
brain injury patients tbi traumatic study cerebral results severe group cognitive clinical pressure imaging following outcome control using children test

Topic #1:
new said york news atlanta like times year service time people undated just music journal constitution city says com years

Topic #2:
patients injury injuries trauma head study results traumatic brain treatment cases patient fractures years case outcome methods clinical tbi surgery

Topic #3:
said year bush percent new enron company president government people economy years million state companies states economic united time billion
```

# 2. [60] Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on trie;
3. Measure suggestion speed;
4. (+) Optionally add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 2.0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [10]:
!pip install pygtrie

  Created wheel for pygtrie: filename=pygtrie-2.4.2-cp37-none-any.whl size=19063 sha256=f88dc748c23a29c3599c48f0a174e04bf2df64c2ba7fbf144997d07e00ea4681
  Stored in directory: /root/.cache/pip/wheels/db/57/91/73782136379fe419036c5ec0e4070d8b3a35f2a36bd6a94ed8
Successfully built pygtrie


### 2.0.1. Check it works and understand the example

In [2]:
import pygtrie
t = pygtrie.CharTrie()

# trie can be considered as a form of organizing a set of map
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

# "this" string is present in a set
n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
# "this" prefix is present in a set
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}\nSubtree = {s}")

# iterate a subtree
for key, val in t.iteritems("this"):
    print(key, '~', val)

CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False
Subtree = True
this is 1 ~ A
this is 2 ~ B


## 2.1. Build a trie upon a dataset

### 2.1.1. [5] Read dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset.

In [3]:
import pandas as pd

aol_data = None

#TODO: Read the dataset, e.g. as pandas dataframe
drivePath = "/content/drive/MyDrive/AIR/"
filename = "user-ct-test-collection-01.txt"

aol_data = pd.read_csv(drivePath + filename, delimiter = "\t", error_bad_lines=False)

aol_data = aol_data.fillna(0)

assert aol_data.shape[0] == 3558411, "Dataset size doesnt' match"

In [4]:
stopWords = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with']

### 2.1.2. [20] Build Trie

We want suggest function to be **non-sensitive to stop words** because we don't want to upset the user if he confuses/omits prepositions, for example. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build Trie based on the dataset, **storing query statistics such as query frequency, urls and ranks in nodes**. Some queries may not have associated urls, others may have multiple ranked urls. Think of the way to store this information.

In [5]:
# aol_trie = pygtrie.CharTrie()
#TODO: build trie based on dataset
# Data = {AnonID, Query, QueryTime, ItemRank, ClickURL}

def createIndex(aol_data):
  aol_trie = pygtrie.CharTrie()
  for idx in range(len(aol_data)):
    nitem = aol_data['Query'][idx]
    queryCheck = 0
    if type(nitem) == str:
      if len(nitem) > 0:
         queryCheck = 1

    url = aol_data['ClickURL'][idx]
    urrl = 1
    if type(url) == str:
      if len(url) > 0:
         urrl = (url, 1)
    
    if queryCheck != 0:
      if aol_trie.has_node(nitem) == 1:
        index = aol_trie[nitem]
        index[0] += 1
        if urrl not in index:
          index.append(urrl)
        aol_trie[nitem] = index
      else:
        if urrl != 1:
          aol_trie[nitem] = [1 , urrl]
        else:
          aol_trie[nitem] = [1]
  return aol_trie

aol_trie = createIndex(aol_data)

# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a list field. But you can do something different. 
    for url in val:
      if isinstance(url, tuple):
        urll = url[0]
        bag.append(urll)
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ [5, ('http://www.surveyconnect.com', 1), ('http://www.custominsight.com', 1), ('http://www.askemployees.com', 1), ('http://www.lg-employers.gov.uk', 1), 1]
sample questions for immigration interview ~ [1]
sample questions for interview ~ [1, ('http://www.quintcareers.com', 1)]
sample questions for family interview ~ [3, ('http://www.grandparents-day.com', 1), ('http://www.quintcareers.com', 1), ('http://jobsearchtech.about.com', 1)]
sample questions for us citizenship test ~ [1, ('http://uscis.gov', 1)]
sample questions sociology race and ethnicity ~ [1]
sample questions biology ~ [2, ('http://www.utexas.edu', 1), ('http://www.troy.k12.ny.us', 1)]
sample questionarie teaching evaluation ~ [1]
sample questionnaire teaching evaluation ~ [5, ('http://www.surveyconsole.com', 1), ('http://www.usask.ca', 1), ('http://teaching.berkeley.edu', 1), ('http://www.flinders.edu.au', 1), ('http://oregonstate.edu', 1)]
sample questionnaire for clinical research coordinators c

## 2.3. [20] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Q: What is the empirical threshold for minimal prefix for suggest?

In [6]:
def complete_user_query(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    newQuery = ""
    res = []
    for word in query.split():
      if word not in stopWords:
         newQuery += word
    
    # iterate a subtree
    try:  
      for key, val in trie.iteritems(newQuery):
          res.append((val[0], key))
        
      res.sort(key=lambda val: val[0], reverse=True)
      
      newres = []
      for r in res:
        newres.append(r[1])

      return newres[:top_k]
    except:
      pass

        
inp = "trie"
print("Query:", inp)
# print("Results:")
res = complete_user_query(inp, aol_trie)
print("Results:", res)
# #NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

Query: trie
Results: ['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', "tried and truechildren's consignment sale"]


## 2.4. [15] Measure suggest speed ##

Check how fast your search is working. Consider changing your code if it takes too long on average.

In [7]:
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure avg execution time (in milliseconds) per query and print it out
import time
totalTime = 0
for query in inp_queries:
  tic1 = time.process_time()
  try:
    res = complete_user_query(query, aol_trie, 1)
    tic2 = time.process_time()
    timediff = tic2-tic1
    totalTime += timediff
  except:
    pass
print("Time taken for %d queries: "%len(inp_queries), totalTime,"\n",(totalTime/len(inp_queries))*1000, "microseconds/query")

Time taken for 11 queries:  0.10319116299996267 
 9.381014818178425 microseconds/query


## 2.5. [+10/100] Bonus task - add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

In [14]:
!pip install pyspellchecker 

     |████████████████████████████████| 2.5MB 6.3MB/s 


In [14]:
from spellchecker import SpellChecker
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]


def complete_user_query_with_spellchecker(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    spell = SpellChecker(language='en')
    bq = []

    nquery = query.split()
    for v in nquery:
      if len(v)>1:
        bq.append(v)

    correctedQueries = []

    misspelled = spell.unknown(bq)
    
    for m in misspelled:
      replacement = spell.correction(m)
      correctedQueries.append(query.replace(m, replacement))
    res = query
    for cor in correctedQueries:   
      res = complete_user_query(cor, aol_trie)
    return res

trie = aol_trie
for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, trie, 5) == \
            complete_user_query_with_spellchecker(q, trie, 5), "Assert {} and {} give different results".format(q, qc)